In [1]:
!pip show tensorflow
!pip install --upgrade tensorflow

Name: tensorflow
Version: 2.4.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wrapt, h5py, termcolor, wheel, astunparse, opt-einsum, protobuf, gast, tensorflow-estimator, keras-preprocessing, typing-extensions, grpcio, tensorboard, numpy, google-pasta, six, absl-py, flatbuffers
Required-by: fancyimpute
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.4.0)


In [1]:
#from __future__ import unicode_literals, print_function, division
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Flatten, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint
from numpy import array, argmax
from numpy.random import rand, shuffle
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
import scipy
import statsmodels
import sklearn
import tensorflow
import keras
from io import open
import unicodedata
import random
import math
import os
import time
import re
import sys
from unicodedata import normalize
from numpy import array
from pickle import dump, load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint

INDIC_NLP_LIB_HOME=r"/content/drive/MyDrive/anoopkunchukuttan-indic_nlp_library-eccde81/src"
INDIC_NLP_RESOURCES=r"/content/drive/MyDrive/indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader

from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

In [3]:
loader.load()


AttributeError: ignored

In [2]:
def load_doc(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()
    return text

def to_pairs(hindi_text, telugu_text):
    hindi_lines = hindi_text.strip().split('\n')
    telugu_lines = telugu_text.strip().split('\n')
    pairs = []
    for i in range(len(telugu_lines)):
        pairs.append([])
        pairs[i].append(pre_process_hindi_sentence(hindi_lines[i]))
        pairs[i].append(pre_process_telugu_sentence(telugu_lines[i]))
    return pairs

def clean_text(text):
    text = text.replace(u',','')
    text = text.replace(u'"','')
    text = text.replace(u'"','')
    text = text.replace(u"‘‘",'')
    text = text.replace(u"’’",'')
    text = text.replace(u"''",'')
    text = text.replace(u"।",'')
    text=text.replace(u',','')
    text=text.replace(u'"','')
    text=text.replace(u'(','')
    text=text.replace(u')','')
    text=text.replace(u'"','')
    text=text.replace(u':','')
    text=text.replace(u"'",'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text= re.sub("'", '', text)
    text=re.sub('[0-9+\-*/.%]', '', text)
    text=text.strip()
    text=re.sub(' +', ' ',text)
    exclude = set(string.punctuation)
    text= ''.join(ch for ch in text if ch not in exclude)
    return text

def pre_process_hindi_sentence(line):
    line=re.sub('[a-zA-Z]', '', line)
    line = clean_text(line)
    remove_nuktas = False
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("hi",remove_nuktas)
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line):
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = ' '.join(line)
    line = 'START_ '+ line + ' _END'
    return (line)

def pre_process_telugu_sentence(line):
    line=re.sub('[a-zA-Z]', '', line)
    line = clean_text(line)
    remove_nuktas = False
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("hi",remove_nuktas)
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line):
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = ' '.join(line)
    line = 'START_ '+ line + ' _END'
    return (line)

hindi_text = load_doc('/content/drive/MyDrive/surya_hindi_telugu/HIN.txt')
telugu_text = load_doc('/content/drive/MyDrive/surya_hindi_telugu/TEL.txt')
pairs = to_pairs(hindi_text, telugu_text)
df = pd.DataFrame(pairs)
df.columns = ["hin", "tel"]
#df

In [3]:
lines = df

In [4]:
all_hin_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hin_words:
            all_hin_words.add(word)


all_telugu_words=set()
for tel in lines.tel:
    for word in tel.split():
        if word not in all_telugu_words:
            all_telugu_words.add(word)

#print (all_hin_words)
print (len(all_hin_words))
print (len(all_telugu_words))

12560
16242


In [5]:
# Max Length of source sequence
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print (max_length_src)

# Max Length of target sequence
lenght_list=[]
for l in lines.tel:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print (max_length_tar)

85
59


In [6]:
input_words = sorted(list(all_hin_words))
target_words = sorted(list(all_telugu_words))
num_encoder_tokens = len(all_hin_words)
num_decoder_tokens = len(all_telugu_words)
num_encoder_tokens, num_decoder_tokens
num_encoder_tokens += 1
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

16243

In [7]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [8]:
# Train - Test Split
X, y = lines.hin, lines.tel
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((9899,), (1100,))

In [9]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [10]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        # range(start, stop, step)
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [11]:
latent_dim = 500
vec_len = 300 # Length of the vector that we willl get from the embedding layer
dropout_rate = 0.2 # Rate of the dropout layers
encoder_inputs = Input(shape=(None,))

#enc_emb =  Embedding(num_encoder_tokens, vec_len, mask_zero = True)(encoder_inputs)
#enc_emb =  Embedding(num_encoder_tokens, vec_len)(encoder_inputs)
enc_emb = Embedding(input_dim = num_encoder_tokens, output_dim = vec_len, mask_zero = True)(encoder_inputs)

encoder_dropout = (TimeDistributed(Dropout(rate = dropout_rate)))(enc_emb)
encoder_lstm1 = LSTM(latent_dim, return_sequences=True)(encoder_dropout)
encoder_lstm2 = LSTM(latent_dim, return_sequences=True)(encoder_lstm1)
encoder_lstm3 = LSTM(latent_dim, return_sequences=True)(encoder_lstm2)
encoder_LSTM4_layer = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM4_layer(encoder_lstm3)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [12]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, vec_len, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_dropout = (TimeDistributed(Dropout(rate = dropout_rate)))(dec_emb)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.


decoder_LSTM_layer = LSTM(latent_dim, return_sequences=True)
decoder_LSTM = decoder_LSTM_layer(decoder_dropout, initial_state = encoder_states)

decoder_LSTM_2_layer = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder_LSTM_2_layer(decoder_LSTM)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [13]:
model.summary()

# Define a checkpoint callback :
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    3768300     input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, None, 300)    0           embedding[0][0]                  
__________________________________________________________________________________________________
lstm (LSTM)                     (None, None, 500)    1602000     time_distributed[0][0]           
______________________________________________________________________________________________

In [14]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 80
epochs = 10

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
123/123 [==============================] - 1757s 14s/step - loss: 1.5797 - acc: 0.0906 - val_loss: 1.4879 - val_acc: 0.1107
Epoch 2/10
123/123 [==============================] - 1729s 14s/step - loss: 1.4203 - acc: 0.1099 - val_loss: 1.4754 - val_acc: 0.1124
Epoch 3/10
123/123 [==============================] - 1713s 14s/step - loss: 1.3720 - acc: 0.1131 - val_loss: 1.4639 - val_acc: 0.1168
Epoch 4/10
123/123 [==============================] - 1707s 14s/step - loss: 1.3347 - acc: 0.1184 - val_loss: 1.4555 - val_acc: 0.1157
Epoch 5/10
123/123 [==============================] - 1705s 14s/step - loss: 1.2969 - acc: 0.1240 - val_loss: 1.4524 - val_acc: 0.1206
Epoch 6/10
123/123 [==============================] - 1708s 14s/step - loss: 1.2612 - acc: 0.1275 - val_loss: 1.4291 - val_acc: 0.1249
Epoch 7/10
123/123 [==============================] - 1718s 14s/step - loss: 1.2251 - acc: 0.1325 - val_loss: 1.4275 - val_acc: 0.1255
Epoch 8/10
123/123 [==============================] - 1

In [15]:
#save the model
model.save_weights('nmt_weights.h5')

In [16]:
#load the model
model.load_weights('nmt_weights.h5')

In [17]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence
decoder_dropout2 = (TimeDistributed(Dropout(rate = dropout_rate)))(dec_emb2)
decoder_LSTM2 = decoder_LSTM_layer(decoder_dropout2, initial_state = decoder_states_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_LSTM_2_layer(decoder_LSTM2)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) #A dense softmax layer to generate prob dist. over the target vocabulary


# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 98):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [20]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input hindi sentence:', X_train[k:k+1].values[0])
print('Actual telugu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted telugu Translation:', decoded_sentence[:-4])

Input hindi sentence: START_ इस मौसम की आर्द्रता और तापमान जीवाणुओं और कीटाणुओं को पनपने में सहायक होते है _END
Actual telugu Translation:  ఈ వాతావరణం యొక్క తేమ మరియు ఉష్ణోగ్రత జీవులు మరియు కీటకాల పెరుగుదలకు సహాయపడుతుంది 
Predicted telugu Translation:  ఈ వ్యాధి తీసుకెళ్లండి 


In [21]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input hindi sentence:', X_train[k:k+1].values[0])
print('Actual telugu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted telugu Translation:', decoded_sentence[:-4])

Input hindi sentence: START_ औरत ऑपरेशन के दौरान पूरे होश में रहती है _END
Actual telugu Translation:  ఆపరేషన్ సమయంలో మహిళ పూర్తి స్పృహలో ఉంటుంది 
Predicted telugu Translation:  అందువల్ల వ్యాధి అర్ధగోళంలో కెరాటోమలాసియా ఈ వ్యాధి తీసుకెళ్లండి 


In [ ]:
print('Input hindi sentence:', X_train[k:k+1].values[0])

Input English sentence: START_ చర్న్ పురుగులు ఇవి తెల్లని తీగ లాగా చాలా చిన్నవి _END


In [22]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input hindi sentence:', X_train[k:k+1].values[0])
print('Actual telugu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted telugu Translation:', decoded_sentence[:-4])

Input hindi sentence: START_ मिलावटी चीजों से न सिर्फ लीवर बल्कि आँतों पर भी असर पड़ता है _END
Actual telugu Translation:  కలుషితమైన విషయాలతో కాలేయం మాత్రమే కాదు దాని ప్రభావం ప్రేగులపై కూడా కనిపిస్తుంది 
Predicted telugu Translation:  మీరు ఆరోగ్య ఇవే ఈ వ్యాధి తీసుకెళ్లండి 


In [23]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input hindi sentence:', X_train[k:k+1].values[0])
print('Actual telugu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted telugu Translation:', decoded_sentence[:-4])

Input hindi sentence: START_ कैंसर के कारण ऐसे अनेक लक्षण दिखलाई पड़ने लगते हैं जो कैंसर होने का आभास देते हैं _END
Actual telugu Translation:  క్యాన్సర్ కారణంగా అనేక లక్షణాలు కనిపిస్తాయి ఇవి క్యాన్సర్ యొక్క ముద్రలను ఇస్తాయి 
Predicted telugu Translation:  ఈ వ్యాధి తీసుకెళ్లండి 


In [24]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input hindi sentence:', X_train[k:k+1].values[0])
print('Actual telugu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted telugu Translation:', decoded_sentence[:-4])

Input hindi sentence: START_ बच्चों के संपूर्ण आहार में कुछ चीजें जैसे फल दूध स्वीट्स सब्जियाँ अगर माँसाहारी है तो मीट अंडे बहुत महत्वपूर्ण हैं _END
Actual telugu Translation:  పిల్లల పూర్తి ఆహారంలో పండ్లు పాలు స్వీట్లు కూరగాయలు మాంసాహారం ఉంటే మాంసం గుడ్లు చాలా ముఖ్యమైనవి 
Predicted telugu Translation:  మీరు ఆరోగ్య ఇవే ఈ వ్యాధి తీసుకెళ్లండి 


In [25]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input hindi sentence:', X_train[k:k+1].values[0])
print('Actual telugu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted telugu Translation:', decoded_sentence[:-4])

Input hindi sentence: START_ मात्र रसौली की उपस्थिति ही उसे निकाले जाने की अनिवार्यता नहीं है _END
Actual telugu Translation:  వాపు గ్రంథుల ఉనికి మాత్రమే దాని తొలగింపుకు అవసరం లేదు 
Predicted telugu Translation:  మీరు ఆరోగ్య ఇవే ఈ వ్యాధి తీసుకెళ్లండి 
